In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../data/input/modified/DOE School Calendar Dates_edited.csv').drop(columns = ['Unnamed: 3'])
data = data.replace('–', '-')

In [3]:
# Data start date should be Sept. 7, 2017

# Have to preprocess date ranges
# Split into start and end dates. If there is a hyphen, 
# take the two numbers on either side of it and use one as start date and one as an end date

# 27-30 Dec 2022

# For this to work, replace all '‚Äì' (big hyphen code) in csv with hyphens '-' 

start_dates, end_dates = [], []
for i in data['Dates']:
    if ('-' not in i):
        start_dates.append(pd.to_datetime(i))
        end_dates.append(pd.to_datetime(i))
    else:
        x = i.split(' ')
        start_dates.append(pd.to_datetime(' '.join([k.split('-')[0] if ('-' in k) else k for k in x])))
        end_dates.append(pd.to_datetime(' '.join([k.split('-')[1] if ('-' in k) else k for k in x])))
        
data['Start Date'] = start_dates
data['End Date'] = end_dates

In [4]:
def get_year(school_year, month):
    if int(month) < 9:
        return int(school_year[:4]) + 1
    else:
        return int(school_year[:4])

In [5]:
data['End Date'] = pd.to_datetime(data['End Date'].dt.month.astype(str) + "/" + data['End Date'].dt.day.astype(str) + "/" + data.apply( (lambda x: get_year(x['School Year'], x['End Date'].month)), axis = 1).astype(str))
data['Start Date'] = pd.to_datetime(data['Start Date'].dt.month.astype(str) + "/" + data['Start Date'].dt.day.astype(str) + "/" + data.apply( (lambda x: get_year(x['School Year'], x['Start Date'].month)), axis = 1).astype(str))

In [6]:
# Adding covid shutdown 3/16/2020 - 9/13/2021

covid = {'School Year': '2021-2022', 
         'Dates': '3/16/2020 - 9/13/2021', 
         'Reason': 'COVID', 
         'Start Date': pd.to_datetime('3/16/2020'), 
         'End Date': pd.to_datetime('9/13/2021')
        }
data = data.append(covid, ignore_index=True)

/var/folders/jt/mzncz72n2l53hxv8vgtmr8xw0000gn/T/ipykernel_83315/3668757566.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(covid, ignore_index=True)


In [7]:
# Sort reasons
data['Closure'] = data['Reason'].apply( lambda x: 1 if  ( ("closed" in x) | 
                                       ("no students" in x) | 
                                       ("9-12 schools only" in x) | 
                                       ("remote" in x) | 
                                       ("hours early" in x) | 
                                       ("do not attend" in x) | 
                                       ("CLOSED" in x) | 
                                       ("Clerical Day" in x) | 
                                       ("staff development" in x) | 
                                       ("Students not in attendance" in x) | 
                                       ("Students will not" in x) | 
                                       ("will not be in attendance" in x) | 
                                       ("COVID" in x) | 
                                       ("Summer Recess" in x) ) else 0 )

In [8]:
data.to_csv('../data/output/school_calendar_dates.csv')